<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: старший преподаватель Факультета Компьютерных Наук ВШЭ, программист-исследователь Mail.ru Group  Юрий Кашницкий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Занятие 3. Обучение с учителем. Методы классификации
## <center>Часть 3. Дерево решений в задаче кредитного скоринга

Решается задача кредитного скоринга. 

Признаки клиентов банка:
- Age - возраст (вещественный)
- Income - месячный доход (вещественный)
- BalanceToCreditLimit - отношение баланса на кредитной карте к лимиту по кредиту (вещественный)
- DIR - Debt-to-income Ratio (вещественный)
- NumLoans - число заемов и кредитных линий
- NumRealEstateLoans - число ипотек и заемов, связанных с недвижимостью (натуральное число)
- NumDependents - число членов семьи, которых содержит клиент, исключая самого клиента (натуральное число)
- Num30-59Delinquencies - число просрочек выплат по кредиту от 30 до 59 дней (натуральное число)
- Num60-89Delinquencies - число просрочек выплат по кредиту от 60 до 89 дней (натуральное число)
- Delinquent90 - были ли просрочки выплат по кредиту более 90 дней (бинарный) - имеется только в обучающей выборке

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Загружаем данные.**

In [2]:
df = pd.read_csv('../../data/credit_scoring_train.csv', 
                       index_col='client_id')

In [3]:
df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit,Delinquent90
client_id,,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028,0
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679,0
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227,0
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930,0
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350,0


In [4]:
y = df['Delinquent90']
X = df.drop('Delinquent90', axis=1)

In [5]:
y.value_counts()

0    69987
1     5013
Name: Delinquent90, dtype: int64

**Посчитаем число пропусков в каждом признаке.**

In [6]:
X.isnull().sum()

DIR                          0
Age                          0
NumLoans                     0
NumRealEstateLoans           0
NumDependents             1916
Num30-59Delinquencies        0
Num60-89Delinquencies        0
Income                   14847
BalanceToCreditLimit         0
dtype: int64

In [7]:
for col in X.columns:
    print(col, X[col].isnull().sum(), sep='\t')

DIR	0
Age	0
NumLoans	0
NumRealEstateLoans	0
NumDependents	1916
Num30-59Delinquencies	0
Num60-89Delinquencies	0
Income	14847
BalanceToCreditLimit	0


**Заменим пропуски медианными значениями.**

In [8]:
X['NumDependents'].fillna(X['NumDependents'].median(), inplace=True)
X['Income'].fillna(X['Income'].median(), inplace=True)

In [9]:
# Результат работы двух строк одинаковый
X['Income'].fillna(X['Income'].median(), inplace=True)
X['Income'] = X['Income'].fillna(X['Income'].median(), inplace=False)

**Разобьем данные на train и test**

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [11]:
X_train.shape, X_test.shape

((52500, 9), (22500, 9))

### Дерево решений без настройки параметров

In [12]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

**Прогноз для тестовой выборки.**

In [13]:
y_predicted = clf.predict(X_test)
y_predicted[:10]

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0])

**Метрика качества**

In [14]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_test, y_predicted)
print(score)

0.891733333333


In [15]:
print('Zeros score: ', accuracy_score(y_test[y_test == 0], y_predicted[y_test == 0]))
print('Ones score: ', accuracy_score(y_test[y_test == 1], y_predicted[y_test == 1]))

Zeros score:  0.936760020921
Ones score:  0.247106875425


## Дерево решений с настройкой параметров с помощью GridSearch

In [16]:
from sklearn.model_selection import StratifiedKFold

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
params = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_leaf': [3, 10, 20, 30]
}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, scoring='accuracy', cv=5)
gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 5, 7, 9], 'min_samples_leaf': [3, 10, 20, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [19]:
gs.best_params_

{'max_depth': 5, 'min_samples_leaf': 30}

In [20]:
gs.best_score_

0.93428571428571427

In [21]:
clf = DecisionTreeClassifier(max_depth=5, min_samples_leaf=30, random_state=42)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=30, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [22]:
y_predicted = clf.predict(X_test)
score = accuracy_score(y_test, y_predicted)
print(score)

0.935244444444
